In [48]:
import os
import cloudpickle
import ray
from ray import tune
from ray.rllib import agents
import torch
from ray.tune.registry import get_trainable_cls

checkpoint = "saved_checkpoint/checkpoint/checkpoint"
run = "PPO"
env = "gym_fishing.envs.FishingCtsEnv"


# Based on rllib evaluate,
# https://github.com/ray-project/ray/blob/master/rllib/evaluate.py
config_dir = os.path.dirname(checkpoint)
config_path = os.path.join(config_dir, "../params.pkl")
config_path
with open(config_path, "rb") as f:
    config = cloudpickle.load(f)
config

{'model': {'use_lstm': True},
 'framework': 'torch',
 'num_gpus': 1,
 'num_workers': 4,
 'env': 'gym_fishing.envs.FishingCtsEnv'}

In [49]:

# Based on rllib evaluate,
# https://github.com/ray-project/ray/blob/master/rllib/evaluate.py
# Make sure worker 0 has an Env.
config["create_env_on_driver"] = True
# Make sure we have evaluation workers.
if not config.get("evaluation_num_workers"):
    config["evaluation_num_workers"] = config.get("num_workers", 0)
if not config.get("evaluation_duration"):
    config["evaluation_duration"] = 1
# Hard-override this as it raises a warning by Trainer otherwise.
# Makes no sense anyways, to have it set to None as we don't call
# `Trainer.train()` here.
config["evaluation_interval"] = 1




# Create the Trainer from config.
cls = get_trainable_cls(run)
agent = cls(env=env, config=config)

agent.restore(checkpoint)

2022-03-06 04:50:48,075	WARNING deprecation.py:45 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-03-06 04:50:48,210	INFO trainable.py:472 -- Restored on 172.18.0.8 from checkpoint: saved_checkpoint/checkpoint/checkpoint
2022-03-06 04:50:48,211	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 250, '_timesteps_total': 1000000, '_time_total': 2036.3581442832947, '_episodes_total': 19192}
(RolloutWorker pid=222871) 2022-03-06 04:50:50,673	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
(RolloutWorker pid=222875) 2022-03-06 04:50:50,701	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the futur

In [50]:
agent.evaluate()

{'evaluation': {'episode_reward_max': 6.638855040073395,
  'episode_reward_min': 6.638855040073395,
  'episode_reward_mean': 6.638855040073395,
  'episode_len_mean': 101.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [6.638855040073395],
   'episode_lengths': [101]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 0.06185559665455538,
   'mean_inference_ms': 0.6857572817334943,
   'mean_action_processing_ms': 0.05300372254614737,
   'mean_env_wait_ms': 0.04398355297013826,
   'mean_env_render_ms': 0.0},
  'off_policy_estimator': {},
  'timesteps_this_iter': 0}}

In [34]:
# Read in an ExperimentAnalysis from a json state file
#results_path = "~/ray_results/ARS/"
results_path = "~/ray_results/fishing-ppo/"

analysis = tune.ExperimentAnalysis(experiment_checkpoint_path=results_path)
#best_trial = analysis.get_best_trial(metric="episode_reward_mean", mode="max")
best_trial = "PPO_gym_fishing.envs.FishingCtsEnv_deefe_00000"
#best_checkpoint = analysis.get_best_checkpoint(trial=best_trial, metric="episode_reward_mean", mode="max")


In [35]:

#config = analysis.get_best_trial(metric="episode_reward_mean", mode="max").last_result["config"]


Could not find best trial. Did you pass the correct `metric` parameter?


AttributeError: 'NoneType' object has no attribute 'last_result'

In [57]:
# extract the hyper-parameter config
config = analysis.get_best_trial(metric="episode_reward_mean", mode="max").last_result["config"]
config.pop("in_evaluation", None)
